In [1]:
import numpy as np
import pandas as pd

# pd.set_option('display.max_colwidth', None) # show complete content of a cells in table
pd.set_option('display.max_columns', None)

# load xlsx and save to feather format
df1 = pd.read_excel(r"test2.xlsx", sheet_name="Firmyears")
df2 = pd.read_excel(r"test2.xlsx", sheet_name="Directorfirms")
# df1.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\firm-years.ftr")
# df2.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\director-firms.ftr")

# load boath dfs
# df1 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\firm-years.ftr")
# df2 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\director-firms.ftr")

In [2]:
# convert float columns to int
float_columns = df1.select_dtypes(include=np.number).columns.tolist()
df1[float_columns] = df1[float_columns].astype("Int64") #Int64 can store missing values as NaN
# df1.dtypes # datatype of columns

df1[15:25]

,Comp_Panel,Year_Panel
15,1290,2014
16,1290,2015
17,1290,2016
18,1290,2017
19,1290,2018
20,1290,2019
21,3040,1999
22,3040,2000
23,3040,2001
24,3040,2002


In [3]:
# convert float columns to int
float_columns = df2.select_dtypes(include=np.number).columns.tolist()
df2[float_columns] = df2[float_columns].astype("Int64") #Int64 can store missing values as NaN
df2.dtypes # datatype of columns

df2.sort_values(by=['CompanyID', 'YearStart'], ignore_index=True, inplace=True)

# df2 = pd.concat([df2, pd.DataFrame(columns=[i for i in range(1,30)])], sort=False)
df2

,CompanyID,DirectorID,YearStart,YearEnd
0,1290,1604538114,1999,2006
1,1290,10271811383,2002,2013
2,1290,4606410273,2005,2007
3,1290,92619811414,2009,2021
4,1290,4908459581,2012,2021
5,1290,5703367266,2016,2021
6,3040,1602368094,2005,2005
7,3040,3660867630,2011,2017
8,3040,730763321,2011,2016
9,3040,648627186,2013,2021


In [4]:
# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
# https://pandas.pydata.org/docs/reference/api/pandas.wide_to_long.html
df3 = df1.merge(df2, how="inner", left_on="Comp_Panel", right_on="CompanyID")
df3

,Comp_Panel,Year_Panel,CompanyID,DirectorID,YearStart,YearEnd
0,1290,1999,1290,1604538114,1999,2006
1,1290,1999,1290,10271811383,2002,2013
2,1290,1999,1290,4606410273,2005,2007
3,1290,1999,1290,92619811414,2009,2021
4,1290,1999,1290,4908459581,2012,2021
...,...,...,...,...,...,...
499,185536,2017,185536,4589610146,2009,2011
500,185536,2018,185536,40489311023,2003,2009
501,185536,2018,185536,4589610146,2009,2011
502,185536,2019,185536,40489311023,2003,2009


In [57]:
df4 = df1.merge(df2, how="left", left_on="Comp_Panel", right_on="CompanyID")
# df4
# 2. Danach filtern auf die Bedingung (Year_panel >= YearStart & Year_panel <= YearEnd)
df4.drop(["CompanyID"], axis=1, inplace=True)

df5 = df4[(df4["Year_Panel"] >= df4["YearStart"]) & (df4["Year_Panel"] <= df4["YearEnd"])]
df5 = df5.rename(columns={"Comp_Panel": "FirmID", "Year_Panel": "year"})
df5 = df5.drop(["YearStart","YearEnd"], axis=1)
# df5.to_excel("out.xlsx", index=False)
df5['helper1'] = 1
df5

,FirmID,year,DirectorID,helper1
0,1290,1999,1604538114,1
6,1290,2000,1604538114,1
12,1290,2001,1604538114,1
18,1290,2002,1604538114,1
19,1290,2002,10271811383,1
...,...,...,...,...
480,185536,2008,40489311023,1
482,185536,2009,40489311023,1
483,185536,2009,4589610146,1
485,185536,2010,4589610146,1


In [58]:
pd.pivot_table(df5, index=["FirmID", "year"], columns=["DirectorID"])

helper1                                                       \
DirectorID  523410315 648627186 730763321 1602368094 1604538114 1611148175   
FirmID year                                                                  
1290   1999       NaN       NaN       NaN        NaN        1.0        NaN   
       2000       NaN       NaN       NaN        NaN        1.0        NaN   
       2001       NaN       NaN       NaN        NaN        1.0        NaN   
       2002       NaN       NaN       NaN        NaN        1.0        NaN   
       2003       NaN       NaN       NaN        NaN        1.0        NaN   
...               ...       ...       ...        ...        ...        ...   
185536 2007       NaN       NaN       NaN        NaN        NaN        NaN   
       2008       NaN       NaN       NaN        NaN        NaN        NaN   
       2009       NaN       NaN       NaN        NaN        NaN        NaN   
       2010       NaN       NaN       NaN        NaN        NaN        NaN   
       2011       NaN       NaN       NaN        NaN        NaN        NaN   

                                                                               \
DirectorID  2072154378 3510305900 3510385900 3510705900 3660867630 4589610146   
FirmID year                                                                     
1290   1999        NaN        NaN        NaN        NaN        NaN        NaN   
       2000        NaN        NaN        NaN        NaN        NaN        NaN   
       2001        NaN        NaN        NaN        NaN        NaN        NaN   
       2002        NaN        NaN        NaN        NaN        NaN        NaN   
       2003        NaN        NaN        NaN        NaN        NaN        NaN   
...                ...        ...        ...        ...        ...        ...   
185536 2007        NaN        NaN        NaN        NaN        NaN        NaN   
       2008        NaN        NaN        NaN        NaN        NaN        NaN   
       2009        NaN        NaN        NaN        NaN        NaN        1.0   
       2010        NaN        NaN        NaN        NaN        NaN        1.0   
       2011        NaN        NaN        NaN        NaN        NaN        1.0   

                                                                               \
DirectorID  4606410273 4908459581 5703367266 5711257355 5924059411 6825168196   
FirmID year                                                                     
1290   1999        NaN        NaN        NaN        NaN        NaN        NaN   
       2000        NaN        NaN        NaN        NaN        NaN        NaN   
       2001        NaN        NaN        NaN        NaN        NaN        NaN   
       2002        NaN        NaN        NaN        NaN        NaN        NaN   
       2003        NaN        NaN        NaN        NaN        NaN        NaN   
...                ...        ...        ...        ...        ...        ...   
185536 2007        NaN        NaN        NaN        NaN        NaN        NaN   
       2008        NaN        NaN        NaN        NaN        NaN        NaN   
       2009        NaN        NaN        NaN        NaN        NaN        NaN   
       2010        NaN        NaN        NaN        NaN        NaN        NaN   
       2011        NaN        NaN        NaN        NaN        NaN        NaN   

                                                                        \
DirectorID  6983739643 10271811383 11760554309 20260811567 40489311023   
FirmID year                                                              
1290   1999        NaN         NaN         NaN         NaN         NaN   
       2000        NaN         NaN         NaN         NaN         NaN   
       2001        NaN         NaN         NaN         NaN         NaN   
       2002        NaN         1.0         NaN         NaN         NaN   
       2003        NaN         1.0         NaN         NaN         NaN   
...                ...         ...         ...         ... 

In [57]:
# df2[:15]
dfr = df2[df2['CompanyID'] == 1290]

# dfr.iloc[1,4] = 3
dfr

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,CompanyID,DirectorID,YearEnd,YearStart
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,1604538114,2006,1999
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,10271811383,2013,2002
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,4606410273,2007,2005
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,92619811414,2021,2009
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,4908459581,2021,2012
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290,5703367266,2021,2016


In [22]:
# for (columnName, columnData) in df1.iteritems():
#     print('Column Name : ', columnName)
#     print('Column Contents : ', columnData.values)

# for i, j in df1.iterrows():
#     print(j)

for row in df1.itertuples(index=False):
    # print(row)
    print(row[0], row[1])

    df_filtered = df2[df2["CompanyID"] == row[0]]

    for rov in df_filtered.itertuples(index=False):
        print(f"Current regarded director: {rov[1]}")
        if rov[2] <= row[1] and rov[3] >= row[1]: # check if year of df1 is in between yearstart and yearend
            print(f"Yes, director {rov[1]} was active in year {row[1]} \n") 
        else:
            print(f"No, director {rov[1]} was not active in year {row[1]} \n")

1290 1999
Current regarded director: 1604538114
Yes, director 1604538114 was active in year 1999 

Current regarded director: 10271811383
No, director 10271811383 was not active in year 1999 

Current regarded director: 4606410273
No, director 4606410273 was not active in year 1999 

Current regarded director: 92619811414
No, director 92619811414 was not active in year 1999 

Current regarded director: 4908459581
No, director 4908459581 was not active in year 1999 

Current regarded director: 5703367266
No, director 5703367266 was not active in year 1999 

1290 2000
Current regarded director: 1604538114
Yes, director 1604538114 was active in year 2000 

Current regarded director: 10271811383
No, director 10271811383 was not active in year 2000 

Current regarded director: 4606410273
No, director 4606410273 was not active in year 2000 

Current regarded director: 92619811414
No, director 92619811414 was not active in year 2000 

Current regarded director: 4908459581
No, director 4908459